In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torch torchvision medmnist
!pip install git+https://github.com/MedMNIST/MedMNIST.git

In [ ]:
!pip install scikit-learn
!pip install scikit-learn==1.3.0
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
import medmnist
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from medmnist import INFO, PathMNIST
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
import seaborn as sns
import os
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
from collections import defaultdict

all_true_labels_by_step_label = defaultdict(lambda: defaultdict(list))  # Store true labels for each step and label
all_predicted_labels_by_step_label = defaultdict(lambda: defaultdict(list))  # Store predicted labels for each step and label

# Load and preprocess dataset (224x224 resolution)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Pretrained model normalization
])

# Define the Model with ResNet18 pretrained
class PathMNISTClassifier(nn.Module):
    def __init__(self, num_classes=9):  # PathMNIST has 9 classes
        super(PathMNISTClassifier, self).__init__()
        self.model = models.resnet18(pretrained=True)

        # Modify the final fully connected layer to match the number of classes in PathMNIST
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)

class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Function to unnormalize an image
def unnormalize(tensor, mean, std):
    """Unnormalizes a tensor image with mean and standard deviation."""
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

In [ ]:
# Specify subfolders and main folder
subfolders_to_classify = [
    "a histopathological image of an area with adipose",
    "a histopathological image of an area with mucus",
    "a histopathological image of an area with cancer-associated stroma",
    "a histopathological image of an area with smooth muscle",
    "a histopathological image of an area with colorectal adenocarcinoma epithelium",
    "a histopathological image of an area with lymphocytes",
    "a histopathological image of an area with debris",
    "a histopathological image of an area with background",
    "a histopathological image of an area with normal colon mucosa"
    ]

main_folder = "/content/drive/MyDrive/pathmnist/images"  # Replace with your main folder path

# Load the best model
model_path = '/content/drive/MyDrive/pathmnist/saved_trained_models/224_classification_model.pth'
model = PathMNISTClassifier(num_classes=9)
model.load_state_dict(torch.load(model_path, weights_only=True))
model.eval()

# Get label names
info = INFO['pathmnist']
label_names = list(info['label'].values())
enable_display = True

label_prefix = "a histopathological image of an area with "

In [ ]:
import random
import textwrap  # Import textwrap for wrapping titles
import matplotlib.font_manager as fm

# Initialize a dictionary to store results for each subfolder
subfolder_results = {}

# run the below classification code only for specific training step or all
SUB_SUBFOLDER_NAME_CRITERIA = ""  # run over all steps
# SUB_SUBFOLDER_NAME_CRITERIA = "step1131"  # run for only this step

font_prop = fm.FontProperties(weight='bold')

# Iterate through subfolders and classify images
for subfolder_name in subfolders_to_classify:
    subfolder_path = os.path.join(main_folder, subfolder_name)

    # Check if the subfolder exists before proceeding
    if not os.path.exists(subfolder_path):
        print(f"Skipping subfolder '{subfolder_name}' as it does not exist.")
        continue  # Skip to the next subfolder

    # Extract the true label by removing the prefix
    true_label = subfolder_name.replace(label_prefix, "")

    # Initialize a list to store results for this subfolder
    subfolder_results[subfolder_name] = []

    # Get a list of all image files in the subfolder
    all_image_files = [f for f in os.listdir(subfolder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Randomly select 10 images
    selected_images = random.sample(all_image_files, min(100, len(all_image_files)))  # Select 10 or fewer if less than 10 images are present

    # Initialize counters for true positives, false negatives, and false positives
    true_positives = 0
    false_negatives = 0
    fp_counts = {label: 0 for label in label_names}

    testsaved_data = []
    image_names = []

    # Classify the selected images
    for image_name in selected_images:
        image_path = os.path.join(subfolder_path, image_name)
        image = Image.open(image_path).convert('RGB')
        testsaved_data.append((image, 0))
        image_names.append(image_name)

    testsaved_loader = DataLoader(CustomDataset(testsaved_data, transform=transform), batch_size=1)
    print(f"\nFolder: {subfolder_name}")

    # Initialize figure and axes for the grid
    num_images = len(testsaved_loader)  # Get the total number of images
    num_rows = (num_images + 4) // 5  # Calculate the number of rows needed
    fig, axes = plt.subplots(num_rows, 5, figsize=(10, num_rows * 3))  # Adjust figsize as needed
    # Flatten the axes array for easier indexing
    axes = axes.flatten()

    # Classify images in the current sub-subfolder
    for i, (saved_image, _) in enumerate(testsaved_loader):
        saved_output = model(saved_image)
        _, saved_predicted = torch.max(saved_output, 1)
        predicted_label_name = label_names[saved_predicted.item()]

        # Compare predicted label with true label
        if predicted_label_name == true_label:
            true_positives += 1
        else:
            false_negatives += 1
            # Increment FP count for the predicted label
            fp_counts[predicted_label_name] += 1

        # print(f"   Image: {image_names[i]}, Predicted Label: {predicted_label_name} ({saved_predicted.item()})")
        # Unnormalize and display the image
        if enable_display:
            mean = [0.485, 0.456, 0.406]
            std = [0.229, 0.224, 0.225]
            unnormalized_image = unnormalize(saved_image.clone().squeeze(), mean, std)
            axes[i].imshow(unnormalized_image.permute(1, 2, 0).clip(0, 1))
            axes[i].axis('off')
            # axes[i].set_title(f"Predicted: {predicted_label_name}")  # Add title if desired
            title = f"Predicted: {predicted_label_name}"
            wrapped_title = "\n".join(textwrap.wrap(title, width=10))  # Adjust width as needed
            axes[i].set_title(wrapped_title, fontsize=9, y=1.0, pad=0, fontproperties=font_prop)  # Adjust fontsize and pad

    if enable_display:
      # Hide any unused subplots
      for j in range(num_images, num_rows * 5):
          axes[j].axis('off')

      # Display the grid
      plt.subplots_adjust(hspace=0.8)
      plt.tight_layout()
      plt.show()

    # Print results for the current sub-subfolder
    print(f"Result:")
    # Since you're not iterating through sub-subfolders, use subfolder_name instead of sub_subfolder_name
    print(f" {subfolder_name} True Positives: {true_positives} False Negatives: {false_negatives}\n")
    # Print false positives for this subfolder
    print(f"False Positives:")
    for label, count in fp_counts.items():
        if count > 0:  # Only print labels with false positives
            print(f"  {label}: {count}")
    # Append results for this sub-subfolder to the list for this subfolder
    # Use subfolder_name as the step since there are no sub-subfolders
    subfolder_results[subfolder_name].append((subfolder_name, true_positives, false_negatives,fp_counts))

In [ ]:
fp_counts_by_label = defaultdict(int)  # Stores FP counts for each label

print(f"Final counts of False Positives:")
for subfolder_name, results in subfolder_results.items():
    true_label = subfolder_name.replace(label_prefix, "")  # Extract true label
    # Since there's only one step (the subfolder itself), get the results directly
    _, true_positives, false_negatives, fp_counts = results[0]

    for predicted_label, count in fp_counts.items():
        if predicted_label != true_label:  # Count as FP if predicted label is not the true label
            fp_counts_by_label[predicted_label] += count  # Accumulate FP counts for the label

# Print results
for label, count in fp_counts_by_label.items():
    print(f"Label: {label}, FP: {count}")

In [ ]:
# Function to calculate precision, recall, and F1-score
def calculate_metrics(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    return precision, recall, f1_score

aggregate_metrics = {'tp': 0, 'fp': 0, 'fn': 0}  # Aggregate metrics for all labels

print(f"METRICS:")
for subfolder_name, results in subfolder_results.items():
    true_label = subfolder_name.replace(label_prefix, "")  # Extract true label
    print(f"Label: {true_label}")

    # Get results directly (no step iteration)
    _, true_positives, false_negatives, _ = results[0]
    local_fp = fp_counts_by_label[true_label]  # Get FP for this label

    precision, recall, f1_score = calculate_metrics(true_positives, local_fp, false_negatives)

    # Update aggregate metrics
    aggregate_metrics['tp'] += true_positives
    aggregate_metrics['fp'] += local_fp
    aggregate_metrics['fn'] += false_negatives

    # Print results (no step information)
    print(f" P: {precision:.4f} R: {recall:.4f} F1: {f1_score:.4f} TP:{true_positives} FP:{local_fp} FN:{false_negatives}")

print("\nAGGREGATE METRICS:")
precision, recall, f1_score = calculate_metrics(aggregate_metrics['tp'], aggregate_metrics['fp'], aggregate_metrics['fn'])
print(f" P: {precision:.4f} R: {recall:.4f} F1: {f1_score:.4f} TP:{aggregate_metrics['tp']} FP:{aggregate_metrics['fp']} FN:{aggregate_metrics['fn']}")